# Advent of Code Day 13

In this problem, a firewall is composed of several different layers.  Each layer is at a unique depth from the starting position and is of some size (called a range).  Each layer has its own security scanner that moves back and forth within its range continuously. Now suppose you insert a packet into the firewall and it progresses through the firewall until it reaches the other side.  In part one, the goal is to compute the severity of the trip (where if you're "caught" by the scanner, meaning you enter the layer when the scanner is positioned at the entry index and the severity is the depth * range) when you get onto a packet at time = 0.  In part two, you learn you can delay the time you get on the packet (though once you're on it, you will progress straight through the layers at the same rate as before).  

This was an interesting problem.  I started in part one with modeling the problem using classes (Firewall, Layer, etc).  I liked that solution and thought it was pretty elegant.  However it proved useless in part 2 because the iteration brute-force solution simply took too long.  If you work at the problem long enough (or if you just happen to see it offhand, which I didn't), you discover that you can determine analytically if a specific layer will catch the packet at any time t based on the formula (t + depth) % 2*(size - 1) != 0.  In other words, if t + depth is a multiple of 2*(size - 1), the layer will catch you.  All you have to do is find the lowest time t where you aren't caught by any layer.  While this still required a bit of brute force to search for the time that satisfies that property, such searching is far faster than the iterative class-based approach I used previously.  I briefly investigated whether you could derive a pure analytic solution for all layers without needing to search at all, and that may be possible, but by the time I figured that out, I knew I'd have the answer anyway.  I have removed my original class-based iteration implementation and left only the analytic one.  

In [ ]:
from utils import read_input

In [ ]:
def load_firewall_config(lines):
    
    def split_line(line):
        t = line.split(':')       
        return (int(t[0].strip()), int(t[1].strip()))
    
    return [split_line(line) for line in lines]    

### AnalyticFirewall

My part one solution partitioned the problem into two different classes (Firewall and Layer).  Due to part two's requirement, I ultimately refactored it into a more analytic solution.  To that end, the AnalyticFirewall takes a firewall config (tuples of (depth, size) loaded from the input file) and simply stores them internally.  Then, you can call severity to compute the severity of a trip that starts at release_time OR you can call will_catch_packet to determine if the firewall will catch a packet released at release_time.  You have to remember that if you have a layer at depth 0, it will never contribute to a severity but it still counts as "catching" the packet.  

In [ ]:
class AnalyticFirewall(object):
    
    def __init__(self, layers):
        self.layers = layers
        
    def severity(self, release_time = 0):
        severities = map(lambda (depth, size): ((depth, size), self._will_catch_packet(depth, size, release_time)), self.layers)
        #print severities
        
        return sum([depth*size for (depth, size), caught in severities if caught])
    
    def will_catch_packet(self, release_time = 0):
        
        return any(map(lambda (depth, size): self._will_catch_packet(depth, size, release_time), self.layers))
   
    def _will_catch_packet(self, depth, size, release_time = 0):
        result = ((release_time + depth) % (2 * (size - 1))) 
        
        return result == 0

In [ ]:
def solve_part_one():
    firewall = AnalyticFirewall(load_firewall_config(read_input('Input/day13.txt')))
    
    severity = firewall.severity()
    
    print 'Severity = {}'.format(severity)

In [ ]:
def solve_part_two():
    firewall = AnalyticFirewall(load_firewall_config(read_input('Input/day13.txt')))

    time = 1
    found = []

    while True:  

        caught = firewall.will_catch_packet(time)

        if not caught:
            found.append(time)
            break

        time += 1           

    print 'You can escape if you wait until t = {}'.format(time)

In [ ]:
solve_part_one()